# Стратифицируем только ручные

In [1]:
import pandas as pd
from collections import Counter
import os


In [2]:
df_manual =pd.read_csv("../../Version1//sensitive_topics/Sensitive_topics_crowdsource_only.csv")

In [3]:
df_manual.columns

Index(['text', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice'],
      dtype='object')

In [4]:
topics = list(df_manual.columns)[1:]
len(topics), topics

(18,
 ['offline_crime',
  'online_crime',
  'drugs',
  'gambling',
  'pornography',
  'prostitution',
  'slavery',
  'suicide',
  'terrorism',
  'weapons',
  'body_shaming',
  'health_shaming',
  'politics',
  'racism',
  'religion',
  'sexual_minorities',
  'sexism',
  'social_injustice'])

In [5]:
is_multilabel = []
multilabel_samples_count = 0
for i,el in df_manual.iterrows():
    current_non_null_topics_count = 0
    for t in topics:
        if el[t] >= 0.5:
            current_non_null_topics_count += 1
    if current_non_null_topics_count > 1:
        multilabel_samples_count += 1
        is_multilabel.append(current_non_null_topics_count)
    elif current_non_null_topics_count == 1:
        is_multilabel.append(1)
    else:
        is_multilabel.append(0)

In [6]:
df_manual['multiclass'] = is_multilabel

In [7]:
df_manual.head(2)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,multiclass
0,"Нееет, родаки обязаны родив личинку дать ей об...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.19,0.0,1.0,1.0,1.0,3
1,баба в мусульманских странах она не имеет прав...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,1.0,1.0,4


In [8]:
df_multi_none = df_manual[df_manual['multiclass'] == 0]
df_multi_mono = df_manual[df_manual['multiclass'] == 1]
df_multi_many = df_manual[df_manual['multiclass'] > 1]

In [10]:
split_train = 0.7
split_val = 0.85

# RANDOM_STATE = 0
# RANDOM_STATE = 1
RANDOM_STATE = 2

In [11]:
def split(df):
    df_shuffled = df.sample(frac=1, random_state = RANDOM_STATE).reset_index(drop=True)

    split_index_train = int(len(df_shuffled)*split_train)
    split_index_val = int(len(df_shuffled)*split_val)

    df_shuffled_train = df_shuffled[:split_index_train]
    df_shuffled_val = df_shuffled[split_index_train:split_index_val]
    df_shuffled_test = df_shuffled[split_index_val:]
    
    return df_shuffled_train, df_shuffled_val, df_shuffled_test

In [12]:
# No topics
df_multi_none_train, df_multi_none_val, df_multi_none_test = split(df_multi_none)

In [13]:
#One label samples
handled_samples = set()
df_multi_mono_train_list = []
df_multi_mono_val_list = []
df_multi_mono_test_list = []
for t in topics:
    df_mono_current_topic = df_multi_mono[df_multi_mono[t] >= 0.5]
    df_mono_current_topic_train, df_mono_current_topic_val, df_mono_current_topic_test = split(df_mono_current_topic)
    
    df_multi_mono_train_list.append(df_mono_current_topic_train)
    df_multi_mono_val_list.append(df_mono_current_topic_val)
    df_multi_mono_test_list.append(df_mono_current_topic_test)
    
df_multi_mono_train = pd.concat(df_multi_mono_train_list)
df_multi_mono_val = pd.concat(df_multi_mono_val_list)
df_multi_mono_test = pd.concat(df_multi_mono_test_list)

In [14]:
## Adjust multi

In [15]:
one_sample_labels = []
for i,el in df_multi_many.iterrows():
    current_el_classes = []
    for t in topics:
        if el[t] > 0.5:
            current_el_classes.append(t)
    current_el_classes = sorted(current_el_classes)
    current_el_classes = '='.join(current_el_classes)
    one_sample_labels.append(current_el_classes)


In [16]:
c = Counter(one_sample_labels)
multisamples_counter_dict = dict(c)
c.most_common(10)

[('pornography=sexual_minorities', 261),
 ('politics=racism=social_injustice', 214),
 ('politics=racism', 192),
 ('body_shaming=pornography', 117),
 ('pornography=sexism', 114),
 ('prostitution=sexism', 91),
 ('politics=weapons', 87),
 ('politics=terrorism', 63),
 ('politics=social_injustice', 62),
 ('offline_crime=politics', 61)]

In [17]:
df_multi_many['multiclass'] = one_sample_labels

<ipython-input-17-71733ff6f017>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_many['multiclass'] = one_sample_labels


In [18]:
df_multi_many_train_list = []
df_multi_many_val_list = []
df_multi_many_test_list = []

total_count = 0

for multi_topics,count in multisamples_counter_dict.items():
    df_multi_current_topic = df_multi_many[df_multi_many['multiclass'] ==multi_topics]
    
    try:
        len(df_multi_current_topic) == count
    except Exception as e:
        print(multi_topics,count, len(df_multi_current_topic))
        
    total_count += count
    if count < 2:
        df_multi_many_train_list.append(df_multi_current_topic)
    else:
        df_multi_current_topic_train, df_multi_current_topic_val, df_multi_current_topic_test = split(df_multi_current_topic)
        df_multi_many_train_list.append(df_multi_current_topic_train)
        df_multi_many_val_list.append(df_multi_current_topic_val)
        df_multi_many_test_list.append(df_multi_current_topic_test)
        
df_multi_many_train = pd.concat(df_multi_many_train_list)
df_multi_many_val = pd.concat(df_multi_many_val_list)    
df_multi_many_test = pd.concat(df_multi_many_test_list) 

In [19]:
len(df_multi_many_train) + len(df_multi_many_val) + len(df_multi_many_test), len(df_multi_many)

(3096, 3096)

In [20]:
len(df_multi_many_train), len(df_multi_many_val), round(len(df_multi_many_val)/ (len(df_multi_many_train) + len(df_multi_many_val)),2)

(2131, 417, 0.16)

In [21]:
necessary_columns = ['text'] + topics

In [22]:
df_multi_mono_train = df_multi_mono_train[necessary_columns]
df_multi_mono_val = df_multi_mono_val[necessary_columns]
df_multi_mono_test = df_multi_mono_test[necessary_columns]

df_multi_none_train = df_multi_none_train[necessary_columns]
df_multi_none_val = df_multi_none_val[necessary_columns]
df_multi_none_test = df_multi_none_test[necessary_columns]

df_multi_many_train = df_multi_many_train[necessary_columns]
df_multi_many_val = df_multi_many_val[necessary_columns]
df_multi_many_test = df_multi_many_test[necessary_columns]

In [23]:
df_train = pd.concat([df_multi_mono_train,df_multi_many_train, df_multi_none_train ])
df_train.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

df_val = pd.concat([df_multi_mono_val,df_multi_many_val,df_multi_none_val ])
df_val.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

df_test = pd.concat([df_multi_mono_test,df_multi_none_test,df_multi_many_test])
df_test.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

In [24]:
## Sanity check

In [25]:
df_assert = pd.concat([df_train, df_val, df_test]) 
len(df_assert['text']) , len(set(df_assert['text']))

(9941, 9938)

In [26]:
df_train.to_csv("./compare_manual_manuallVSsemiauto/train_manual_only_rst_{}.csv".format(RANDOM_STATE), index = None)
df_val.to_csv("./compare_manual_manuallVSsemiauto/val_manual_only_rst_{}.csv".format(RANDOM_STATE), index = None)
df_test.to_csv("./compare_manual_manuallVSsemiauto/test_manual_only_rst_{}.csv".format(RANDOM_STATE), index = None)

# Добавляем к трейну все остальное

In [27]:
df_all = pd.read_csv("sensitive_topics.csv")
len(df_all)

33303

In [28]:
manual_set = set(df_manual['text'])

In [29]:
df_all_semi_automative = df_all[~df_all.text.isin(manual_set)]
len(df_all_semi_automative)

26186

In [30]:
df_train_with_semiauto = pd.concat([df_train, df_all_semi_automative])

In [31]:
len(df_train_with_semiauto)

33100

In [32]:
df_train_with_semiauto.to_csv("./compare_manual_manuallVSsemiauto/train_manualVSsemiauto_rst_{}.csv".format(RANDOM_STATE), index = None)
